In [1]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import matplotlib.pyplot as plt
import librosa
import librosa.display
from tqdm import tqdm
import json
import os
import pickle
import sys
from pathlib import Path
from sklearn.model_selection import train_test_split
import soundfile as sf
from utils import GRU

In [2]:
f = '../../../../Music Technology/Datasets/musdb18hq/'
os.listdir(f)

['.DS_Store',
 'chunks_vocal',
 'test',
 'train_data.pt',
 'test_data.pt',
 'chunks_chord',
 'model_train',
 'train',
 'model_test']

## Creating the chunks

In [3]:
SR = 44100
HOP = 256
FRAMES = 6

In [4]:
chord_detector = GRU()
chord_detector.load_state_dict(torch.load('./models/chord_detector.pth'))
chord_detector.eval()

GRU(
  (gru): GRU(12, 256, num_layers=2, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=512, out_features=12, bias=True)
)

In [5]:
def predict(model, audio, chroma_req = True, chord_templates:dict = json.load(open('./chord_templates.json')), sr = SR, hop = HOP):
    if chroma_req:
        chroma = torch.Tensor(librosa.feature.chroma_cens(y=audio, sr = sr, hop_length=hop)).T.unsqueeze(0)
    else:
        chroma = audio
    with torch.no_grad():
        outputs = nn.functional.softmax(model(chroma), 1)[0]
    min_val = 120
    min_key = ''
    for key, val in chord_templates.items():
        out = torch.norm(torch.Tensor(val) - outputs)
        if min_val >= out:
            min_val = out
            min_key = key
    return min_key

In [6]:
def prediction(model, chroma, frame = 6):
    stack = []
    time = []
    model.eval()
    pred = predict(model, chroma[:frame, :].unsqueeze(0), False)
    prev_pred = pred
    dur = 1
    main_sub = 0
    for i in tqdm(range(frame, chroma.shape[0]-frame+1, frame)):
        model.eval()
        pred = predict(model, chroma[i:i+frame, :].unsqueeze(0), False)
        if(pred != prev_pred):
            if(dur>10):
                if(len(stack)==0):
                    stack.append(prev_pred)
                elif(stack[-1]==prev_pred):
                    dur = 0
                    prev_pred = pred
                    continue
                else:
                    stack.append(prev_pred)
                if len(time)!=0:
                    time.append((i)*HOP/SR - main_sub)
                else:
                    main_sub = (i)*HOP/SR
                    time.append(0.0)
            dur = 0
            prev_pred = pred
        dur+=1
    return stack, time

In [ ]:
# # !!! Code in this cell is for chunking the dataset into little bits - you'll only need to run this once

# 

# data_location = "../../../../Music Technology/Datasets/musdb18hq/test/"
# out_location = "../../../../Music Technology/Datasets/musdb18hq/"
# folders = os.listdir(data_location)
# count = 0

# # print(chord_templates["F#"])

# for folder in folders:
#     if not os.path.isdir(data_location+folder):
#         continue
#     mixture_y, _ = librosa.load(data_location + '/' + folder + '/mixture.wav', sr=SR)
#     vocals_y, _ = librosa.load(data_location + '/' + folder + '/vocals.wav', sr=SR)
#     mixture_y = mixture_y/np.max(np.abs(mixture_y))
#     vocals_y = vocals_y/np.max(np.abs(vocals_y))

#     mixture_chroma = torch.Tensor(librosa.feature.chroma_cens(y=mixture_y, sr = SR, hop_length=HOP)).T
#     # vocals_chroma = torch.Tensor(librosa.feature.chroma_cens(y=vocals_y, sr = SR, hop_length=HOP)).T
#     chunk_length = FRAMES
#     nchunks = mixture_chroma.shape[0] // chunk_length # no padding

#     if not os.path.isdir(out_location+'chunks_chord_test/'):
#         os.mkdir(out_location+'chunks_chord_test')
#     if not os.path.isdir(out_location+'chunks_vocal_test/'):
#         os.mkdir(out_location+'chunks_vocal_test')
    
#     # Get chords from mixture chroma
#     chord_stack, time = prediction(chord_detector, mixture_chroma)
#     frame_num = np.array([int(i/((HOP/SR)*6)) for i in time])
#     chord_stack = np.array([frame_num, chord_stack]).T
#     chords = []
#     for prev, curr in zip(chord_stack[:-1], chord_stack[1:]):
#         frame_diff = int(curr[0]) - int(prev[0])
#         chords.extend([prev[1] for _ in range(frame_diff)])
#     chords.extend([chord_stack[-1][1] for _ in range(nchunks - len(chords))])
#     with open(out_location+"chunks_chord_test/chord_"+str(count), "wb") as fp:
#         pickle.dump(chords, fp)
#     print(len(chords), nchunks)
#     sf.write(out_location + 'chunks_vocal_test/vocal_' + str(count)+'.wav', vocals_y, SR)
    
#     # for i in range(0, mixture_chroma.shape[0]-chunk_length+1,chunk_length):
#     #     vocal_chunk = vocals_chroma[i:i+chunk_length,:]
#     #     print(vocal_chunk.shape)\
#         # torch.save(vocal_chunk, out_location + 'chunks_vocal/vocal_' + str(count) + '_' + str(i//chunk_length) + '.pt')
#     count+=1

In [48]:
# # !!! Code in this cell is for chunking the dataset into little bits - you'll only need to run this once

# from pathlib import Path

# data_location = "../../../../Music Technology/Datasets/musdb18hq/train/"
# out_location = "../../../../Music Technology/Datasets/musdb18hq/"
# folders = os.listdir(data_location)
# count = 0

# # print(chord_templates["F#"])

# for folder in folders:
#     if not os.path.isdir(data_location+folder):
#         continue
#     mixture_y, _ = librosa.load(data_location + '/' + folder + '/mixture.wav', sr=SR)
#     vocals_y, _ = librosa.load(data_location + '/' + folder + '/vocals.wav', sr=SR)
#     mixture_y = mixture_y/np.max(np.abs(mixture_y))
#     vocals_y = vocals_y/np.max(np.abs(vocals_y))

#     mixture_chroma = torch.Tensor(librosa.feature.chroma_cens(y=mixture_y, sr = SR, hop_length=HOP)).T
#     # vocals_chroma = torch.Tensor(librosa.feature.chroma_cens(y=vocals_y, sr = SR, hop_length=HOP)).T
#     chunk_length = FRAMES
#     nchunks = mixture_chroma.shape[0] // chunk_length # no padding

#     if not os.path.isdir(out_location+'chunks_chord/'):
#         os.mkdir(out_location+'chunks_chord')
#     if not os.path.isdir(out_location+'chunks_vocal/'):
#         os.mkdir(out_location+'chunks_vocal')
    
#     # Get chords from mixture chroma
#     chord_stack, time = prediction(chord_detector, mixture_chroma)
#     frame_num = np.array([int(i/((HOP/SR)*6)) for i in time])
#     chord_stack = np.array([frame_num, chord_stack]).T
#     chords = []
#     for prev, curr in zip(chord_stack[:-1], chord_stack[1:]):
#         frame_diff = int(curr[0]) - int(prev[0])
#         chords.extend([prev[1] for _ in range(frame_diff)])
#     chords.extend([chord_stack[-1][1] for _ in range(nchunks - len(chords))])
#     with open(out_location+"chunks_chord/chord_"+str(count), "wb") as fp:
#         pickle.dump(chords, fp)
#     print(len(chords), nchunks)
#     sf.write(out_location + 'chunks_vocal/vocal_' + str(count)+'.wav', vocals_y, SR)
    
#     # for i in range(0, mixture_chroma.shape[0]-chunk_length+1,chunk_length):
#     #     vocal_chunk = vocals_chroma[i:i+chunk_length,:]
#     #     print(vocal_chunk.shape)\
#         # torch.save(vocal_chunk, out_location + 'chunks_vocal/vocal_' + str(count) + '_' + str(i//chunk_length) + '.pt')
#     count+=1

100%|██████████| 5708/5708 [00:08<00:00, 685.72it/s]


5709 5709


100%|██████████| 7017/7017 [00:10<00:00, 668.02it/s]


7018 7018


100%|██████████| 6351/6351 [00:09<00:00, 683.89it/s]


...

7855 7855


100%|██████████| 4670/4670 [00:06<00:00, 720.92it/s]


4671 4671


100%|██████████| 2186/2186 [00:03<00:00, 709.51it/s]

2187 2187


In [63]:
class MelChordDataset(Dataset):
    def __init__(
            self, 
            data_location = "../../../../Music Technology/Datasets/musdb18hq/chunks_vocal/",
            out_location = "../../../../Music Technology/Datasets/musdb18hq/chunks_chord/",
            frames_per_chord = 6
        ):
        super(MelChordDataset).__init__()
        self.frames_per_chord = frames_per_chord
        vocals_y = []
        vocals_chroma = []
        chord_templates:dict = json.load(open('./chord_templates.json'))
        act_chord_data = []

        for i in range(len(os.listdir(data_location))): # 100
            with open(out_location+"chord_"+str(i), "rb") as fp:
                chord_data = pickle.load(fp)
            act_chord_data.append(torch.Tensor(np.array([np.array(chord_templates[i]) for i in chord_data])))
            vocals_y.append(librosa.load(data_location + 'vocal_'+str(i)+'.wav', sr=SR)[0])
            vocals_chroma.append(torch.Tensor(librosa.feature.chroma_cens(y=vocals_y[-1], sr = SR, hop_length=HOP)).T)
        
        # act_chord_data[i]: Shape: (num_chords[i], 12)
        # vocals_chroma[i]: Shape: (num_frames[i], 12)
        # num_chords[i] = (num_frames[i] // frames_per_chord)

        self.data = []
        self._create_data(act_chord_data, vocals_chroma)
    
    def _create_data(self, chord_data, chroma_data):
        for (chroma, chords) in zip(chroma_data, chord_data):
            for i in range(0, chroma.shape[0]-self.frames_per_chord, self.frames_per_chord):
                block_chroma = chroma[i:i+self.frames_per_chord,:]
                block_chord = chords[i//self.frames_per_chord]
                if(block_chroma.any()):
                    self.data.append((block_chroma, block_chord))

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index]

In [64]:
train_data = MelChordDataset()
test_data = MelChordDataset(
    data_location="../../../../Music Technology/Datasets/musdb18hq/chunks_vocal_test/",
    out_location="../../../../Music Technology/Datasets/musdb18hq/chunks_chord_test/"
)
torch.save(train_data, './data/final/train_data.pt')
torch.save(test_data, './data/final/test_data.pt')

In [65]:
print(len(train_data))
print(len(test_data))

576514
297423


In [32]:
class GRU(nn.Module):
    def __init__(self, input_size = 12, hidden_size = 64, num_layers = 1, num_classes = 12, bidirectional = True) -> None:
        super(GRU, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.bidirectional = bidirectional

        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first = True, bidirectional=bidirectional)
        if(bidirectional):
            self.fc = nn.Linear(hidden_size*2, num_classes)
        else:
            self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        if(self.bidirectional):
            h0 = torch.zeros(2*self.num_layers, x.size(0), self.hidden_size)
        else:
            h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        out, _ = self.gru(x, h0)
        out = out[:,-1,:] # Since we only want the output of the last cell
        out = self.fc(out)
        return(out)


In [67]:
class Predictor(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.f_dim = 19
        
        self.conv1 = nn.Conv1d(12, 24, 5, 2)
        self.conv2 = nn.Conv1d(24, 48, 5, 2)
        self.conv3 = nn.Conv1d(48, 12, 5, 2)
        self.FC = nn.Linear(self.f_dim * 12, 24)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax()
        
    def forward(self, x):
        h1 = self.relu(self.conv1(x))
        h2 = self.relu(self.conv2(h1))
        h3 = self.relu(self.conv3(h2))
        flat = torch.flatten(h3, 1)
        h4 = self.FC(flat)
        return self.softmax(h4) 


In [74]:
def truth_label_to_int(gt):
    template = json.load(open('./chord_templates.json'))
    int_gt = torch.zeros(len(gt))
    for i in range(len(gt)):
        idx = list(template.keys()).index(gt[i])
        int_gt[i] = idx
    return int_gt.long()
    
def train():
    dataset = Musdb18Dataset('../../Datasets/musdb18-wav')
    train_loader = torch.utils.data.DataLoader(dataset,
                                               batch_size=8,
                                               shuffle=True)
    pred = Predictor()
    adam = torch.optim.Adam(pred.parameters(), lr=0.0001)
    loss_fn = nn.CrossEntropyLoss()
    
    for epoch in range(100):
        epoch_loss = 0
        count = 0
        for i, (X, y) in enumerate(train_loader):
            ground_truth = truth_label_to_int(y)
            out = pred(X)
            loss = loss_fn(out, ground_truth)
            loss.backward()
            adam.step()
            adam.zero_grad()
            epoch_loss += loss.detach()
            count += 1
            print(loss.detach())
        print(epoch_loss / count)
        
    torch.save(pred.state_dict(), 'models/predictor.model')

In [75]:
train()

/var/folders/pq/fg92p7r56cd82gxyhj7r9s140000gn/T/ipykernel_24541/790191051.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.softmax(h4)


tensor(3.1787)
tensor(3.1783)
tensor(3.1783)
tensor(3.1782)
tensor(3.1793)
tensor(3.1782)
tensor(3.1780)
tensor(3.1789)
tensor(3.1784)
tensor(3.1782)
tensor(3.1790)
tensor(3.1774)
tensor(3.1791)
tensor(3.1780)
tensor(3.1787)
tensor(3.1785)
tensor(3.1775)
tensor(3.1788)
tensor(3.1779)
tensor(3.1780)
tensor(3.1784)
tensor(3.1776)
tensor(3.1784)
tensor(3.1796)
tensor(3.1781)


KeyboardInterrupt: 